In [1]:
import numpy as np
import pandas as pd
import sys,os
import random
import copy
from time import time

import matplotlib.pyplot as plt
import seaborn as sns


from utils.method import read_bic_table

from utils.eval import make_ref_groups
from utils.eval import calculate_perfromance, compare_gene_clusters

# 1. Reading expressions and annotations

In [2]:
exprs_file_t = "data/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv"
exprs_t= pd.read_csv(exprs_file_t,sep = "\t",index_col=0)

exprs_file_m = "data/preprocessed_v6/METABRIC_1904_17Kgenes.log2_exprs_z_v6.tsv"
exprs_m= pd.read_csv(exprs_file_m,sep = "\t",index_col=0)

m_subtypes = pd.read_csv("data/preprocessed_v6/METABRIC_1904_17Kgenes.subtypes_and_signatures_v6.tsv",sep = "\t",index_col=0)
m_annotation = pd.read_csv("data/preprocessed_v6/METABRIC_1904.annotation_v6.tsv",sep = "\t",index_col=0)

t_subtypes = pd.read_csv("data/preprocessed_v6/TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv",sep = "\t",index_col=0)
t_annotation = pd.read_csv("data/preprocessed_v6/TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv",sep = "\t",index_col=0)

## 1.1 Preparing ground truth samples sets for performance evaluation

### Example of known_groups dictionary for TCGA-BRCA

*make_ref_groups(subtypes, annotation, exprs)*

**input:**
  - subtypes - subtypes dataframe
  - annotation - annotation dataframe
  - exprs - expression dataframe
  
**returns:**
  -  known_groups = {classificaton1:{"subt1":{s1,s2,...} , "subt2":{...}, "subt3":{...}, ...}, "classi2":{"subtA":{...}}, ... }
*known_groups* is a dictionary with known sample classifications. Each classification (e.g. PAM50 or IHC or Luminal) is a dict that can conatain one or several sample sets 
  -  all_samples = {} set of all samples in expression and annotation files; necessary for computing overlap p-values

In [ ]:
known_groups_t, all_samples_t = make_ref_groups(t_subtypes, t_annotation,exprs_t)
known_groups_m, all_samples_m = make_ref_groups(m_subtypes, m_annotation,exprs_m)

# Example 1: 
## The sructure of known_groups dict for TCGA-BRCA:

We calculate performance for **classifications**:
    * PAM50 = [Luminal, Basal, Her2, Normal]
    * Intrinsic = [Luminal, Basal, Her2, Normal, Claudin-low]
    * PAM50_AB =  [LumA, LumB, Basal, Her2, Normal]
    * SCMOD2 = [ER-/HER2-, ER+/HER2- High Prolif, ER+/HER2- Low Prolif,  HER2+]
    * IHC = [IHC_HER2, IHC_ER, IHC_PR, IHC_TNBC]
And for **isolated sample sets** corresponding to Luminal, Basal, LumA, NEC subtypes etc. 

In [ ]:
for cl in known_groups_t.keys():
    if len(known_groups_t[cl].keys())>1:
        print("classification", cl)
        print("\tsbtypes:"," ".join(known_groups_t[cl].keys()))
    else:
        print(" classification", cl, "(individual subtype)")

# Example 2: 
## evaluation of the resulting sample set (on the example of UnPaSt file) 
reading the results 

In [ ]:
file = "results_on_real_data_WGCNA2/TCGA.seed=670487.bin=kmeans,pval=0.01,clust=WGCNA,direction=UP-DOWN,ds=3,dch=0.995,max_power=10,precluster=True.biclusters.tsv"
result = read_bic_table(file) # reading UnPaSt outputs
print("sample clusters: ", result.shape[0])
# drop clusters too small with < 5 samples
result = result.loc[result["samples"].apply(lambda x: len(x))>=5,:]
print("sample clusters: ", result.shape[0])
result.head(2)

* ensure that results file is a dataframe with "samples" column
* each row in samples column must contain a non-empty set of samples
## performance evaluation
* requires *known_groups* dict and *all_samples* set  
     - using *make_ref_groups()* is recommened for this breast cancer analysis
     - alternatively, *known_groups* dict and *all_samples* can be created manually
* if samples in (bi)clusters do not match *all_samples* set, trho

*calculate_perfromance(bi_clusters_df, annotation, exprs)*

**input:**
  - bi_clusters_df - a dataframe with sample clusters (sets in "sample" column)
  - *known_groups* is a dictionary with known sample classifications. Each classification (e.g. PAM50 or IHC or Luminal) is a dict that can conatain one or several sample sets 
  - *all_samples* = {} set of all samples in expression and annotation files; necessary for computing overlap p-values
  
**returns:**
  - performances - *pandas.Series* with overall perforamnce for each classification from *known_groups* 
  - best_matches - a dataframe with information about the best matching (bi)cluster for each sample set from *known_groups* (helpful for debugging and validation)

In [ ]:
performances, best_matches = calculate_perfromance(result, known_groups_t,all_samples_t)
performances

In [ ]:
best_matches

# 2. Evaluation of the results obtained with different parameters
(UnPaSt)

In [ ]:
# selecting 5 seeds for probabilistic methods 
n_runs = 5
seeds = []
random.seed(42)
for i in range(n_runs):
    seeds.append(random.randint(0,1000000))
print("generate ",n_runs," seeds",seeds)

In [ ]:
subt_t = [] # Perfoemances for TCGA-BRCA
subt_m = [] # Perfoemances for METABRIC
clustering_similarities = [] # Similarities of gene clusters found in TCGA and METABRIC

# UnPaSt parameters 
from run_unpast import run
rpath="/home/olya/anaconda3/envs/r4_env/bin/"
out_dir= "results_on_real_data_WGCNA2/"
basename_t = "TCGA"
basename_m = "METABRIC" 
pvals = [0.05,0.01,0.005,0.001]
bin_methods = ["kmeans","GMM","ward"] 
directions =  [["UP","DOWN"],["BOTH"]]

Because UnPaSt parameters are different for Louvain and WGCNA feature clusterings, 
we run it for each clust_method in a separate *for* loop

In [ ]:
### Louvain 
out_dir= "results_on_real_data_WGCNA2//"
modularities = [0,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

subt_t = []
subt_m = []
clustering_similarities = []


for pval in pvals:
    for bin_method in bin_methods:
        for d in directions:
            for m in modularities:
                # save parameters as a ;-separated string
                params = "bin="+bin_method+";pval="+str(pval)
                params += ";clust="+"Louvain"+";direction="+"-".join(d)+";m="+str(m)
                print()
                for r in range(n_runs):
                    seed = seeds[r]
                    params_dict = {"parameters":params, "seed":seed,"run":r}
                    ### running TCGA or reading results
                    try:
                        t0 = time()
                        fname = out_dir+basename_t+".seed="+str(seed)+\
                        ".bin="+bin_method +",pval="+str(pval)+",clust=Louvain"+",direction="+"-".join(d)+",m="+str(m)+".biclusters.tsv"
                        result_t = read_bic_table(fname)
                        """result_t = run_DESMOND(exprs_file_t, basename_t, out_dir=out_dir,
                                                    save=True, load = True,
                                                    ceiling = 3,
                                                    min_n_samples = 5,
                                                    bin_method = bin_method, pval = pval,
                                                    clust_method = "Louvain",
                                                    similarity_cutoffs = similarity_cutoffs,
                                                    seed = seed,
                                                    verbose = False)
                                                    """
                        time_t = time()-t0
                        # find the best matches between TCGA biclusters and subtypes
                        # and calculate overall performance == weighted sum of Jaccard indexes
                        performance_t,bm_dict_t = calculate_perfromance(result_t, known_groups_t,all_samples_t,
                                                                        performance_measure="ARI")
                        performance_t = performance_t.to_dict()
                        performance_t.update(params_dict)
                        performance_t["time"] = time_t
                        subt_t.append(performance_t)
                        t_failed = False
                    except:
                        print("TCGA biclustering failed with ",seed,  pval,bin_method ,file = sys.stderr)
                        print(fname)
                        t_failed = True
                        subt_t.append({params_dict})

                    ### running METABRIC or reading results
                    try:
                        t0 = time()
                        fname = out_dir+basename_m+".seed="+str(seed)+\
                        ".bin="+bin_method +",pval="+str(pval)+",clust=Louvain"+",direction="+"-".join(d)+",m="+str(m)+".biclusters.tsv"
                        result_m = read_bic_table(fname)
                        """result_m = run_DESMOND(exprs_file_m, basename_m, out_dir=out_dir,
                                                    save=True, load = True,
                                                    ceiling = 3,
                                                    min_n_samples = 5,
                                                    bin_method = bin_method, pval = pval,
                                                    clust_method = "Louvain",
                                                    similarity_cutoffs = similarity_cutoffs,
                                                    seed = seed,
                                                    verbose = False)"""
                        time_m = time()-t0
                        # find the best matches between METABRIC biclusters and subtypes
                        # and calculate overall performance == weighted sum of Jaccard indexes
                        performance_m,bm_dict_m = calculate_perfromance(result_m, known_groups_m,all_samples_m,
                                                                        performance_measure="ARI")
                        performance_m = performance_m.to_dict()
                        performance_m.update(params_dict)
                        performance_m["time"] = time_m
                        subt_m.append(performance_m)
                        m_failed = False
                    except:
                        print("METABRIC biclustering failed with ",seed,  pval,bin_method ,file = sys.stderr)
                        print(fname)
                        m_failed = True
                        subt_m.append(params_dict)
                    print(params,seed, round(performance_t["PAM50"],3),round(performance_m["PAM50"],3))    
                    # compare clustering results - only if gene sets are defined for each cluster
                    if not (t_failed or m_failed): 
                        N = exprs_m.shape[0]
                        clust_sim, bm, bm2 = compare_gene_clusters(result_t,result_m, N)                    
                    else:
                        clust_sim = {}
                    clust_sim.update(params_dict)
                    clustering_similarities.append(clust_sim)


In [ ]:
clust_methods = ["WGCNA"]
dss = [0,1,2,3]
dchs = [0.95,0.995]
cseed = 0

pc = True

for pval in pvals:
    for ds in dss:
        for dch in dchs:
            for d in directions:
                for clust_method in clust_methods:
                    for bin_method in bin_methods:
                        # save parameters as a ;-separated string
                        params = "bin="+bin_method+";pval="+str(pval)+";direction="+str("-".join(d))
                        params += ";clust="+clust_method+";dch="+str(dch)+";ds="+str(ds)+";preClustering=T"
                        print(params)
                        biclusters_t = []
                        biclusters_m = []
                        for r in range(n_runs):
                            seed = seeds[r]
                            #print("run",run,bin_method,pval,m,seed)
                            params_dict = {"parameters":params, "seed":seed,"run":r}

                            ### running TCGA or reading results
                            try:
                                t0 = time()
                                fname = out_dir+basename_t+".seed="+str(seed)+".bin="+bin_method +",pval="+str(pval)+",clust=WGCNA,direction="+str("-".join(d))+",ds="+str(ds)+",dch="+str(dch)+",max_power=10,precluster=True"+".biclusters.tsv"
                                try:
                                    result_t = read_bic_table(fname)
                                except:
                                    print("not found")
                                    """result_t = run(exprs_file_t, basename_t, out_dir=out_dir,
                                                                save=True, load = True,
                                                                min_n_samples = 5,
                                                                bin_method = bin_method, pval = pval,
                                                                directions = d,
                                                                clust_method = clust_method,
                                                                precluster=pc,
                                                                ds=ds,dch=dch,
                                                                rpath=rpath,
                                                                seed = seed,
                                                                verbose = False)
                                                                
                                    """
                                performance_t,bm_dict_t = calculate_perfromance(result_t, known_groups_t,
                                                                                all_samples_t,
                                                                                performance_measure="ARI")
                                performance_t = performance_t.to_dict()
                                performance_t.update(params_dict)
                                performance_t["time"] = time_t
                                subt_t.append(performance_t)
                                t_failed = False
                            except:
                                print("TCGA biclustering failed with ",seed,  pval,bin_method ,file = sys.stderr)
                                print(fname)
                                t_failed = True
                                subt_t.append({params_dict})
                            
                            ### running METABRIC or reading results
                            try:
                                t0 = time()
                                fname = out_dir+basename_m+".seed="+str(seed)+".bin="+bin_method +",pval="+str(pval)+",clust=WGCNA,direction="+str("-".join(d))+",ds="+str(ds)+",dch="+str(dch)+",max_power=10,precluster=True"+".biclusters.tsv"
                                try:
                                    result_m = read_bic_table(fname)
                                except:
                                    print(fname)
                                    """result_m = run(exprs_file_m, basename_m, out_dir=out_dir,
                                                                save=True, load = True,
                                                                min_n_samples = 5,
                                                                bin_method = bin_method, pval = pval,
                                                                directions = d,
                                                                clust_method = clust_method,
                                                                precluster=pc,
                                                                ds=ds,dch=dch,
                                                                rpath=rpath,
                                                                seed = seed,
                                                                verbose = False)
                                    """
                                time_m = time()-t0
                                # find the best matches between METABRIC biclusters and subtypes
                                # and calculate overall performance == weighted sum of Jaccard indexes
                                performance_m,bm_dict_m = calculate_perfromance(result_m, known_groups_m,all_samples_m,
                                                                                performance_measure="ARI")
                                performance_m = performance_m.to_dict()
                                performance_m.update(params_dict)
                                performance_m["time"] = time_m
                                subt_m.append(performance_m)
                                m_failed = False
                            except:
                                print("METABRIC biclustering failed with ",seed,  pval,bin_method ,file = sys.stderr)
                                print(fname)
                                m_failed = True
                                subt_m.append(params_dict)
                            print(params,seed, round(performance_t["PAM50"],3),round(performance_m["PAM50"],3))    
                            # compare clustering results - only if gene sets are defined for each cluster
                            if not (t_failed or m_failed): 
                                N = exprs_m.shape[0]
                                clust_sim, bm, bm2 = compare_gene_clusters(result_t,result_m, N)                    
                            else:
                                clust_sim = {}
                            clust_sim.update(params_dict)
                            clustering_similarities.append(clust_sim)

### Saving method performaces for all parameter combinations

In [ ]:
pd.DataFrame.from_records(clustering_similarities).to_csv("UnPaSt_similarities.tsv",sep = "\t")
pd.DataFrame.from_records(subt_t).to_csv("UnPaSt_TCGA.tsv",sep = "\t")
pd.DataFrame.from_records(subt_m).to_csv("UnPaSt_METABRIC.tsv",sep = "\t")


## 3. Selecting parameters for TCGA and METABRIC
* max. performance for PAM50 classification
* TCGA-BRCA = 0.75
* METABRIC = 0.76

### 4. Optimal parameters selection

* minimal rank sum for TCGA and METABRIC
* slightly inferior compared to "best" parameter combination for each dataset

In [5]:
ds1 = "TCGA-BRCA"
ds2 = "METABRIC"
method = "UnPaSt"
performance_col = "PAM50"

df1 = pd.read_csv("UnPaSt_TCGA.tsv",sep = "\t",index_col =0)
df2 = pd.read_csv("UnPaSt_METABRIC.tsv",sep = "\t",index_col =0)


#if "seed" in df1.columns or "seed" in df2.columns:
df1 = df1.groupby("parameters").agg("mean")
df2 = df2.groupby("parameters").agg("mean")

df1 = df1.sort_values(by=performance_col,ascending= False)
df2 = df2.sort_values(by=performance_col,ascending= False)

df1["rank"] =df1[performance_col].rank(ascending= False)
df2["rank"] =df2[performance_col].rank(ascending= False)
mean_ranks = (df1["rank"]+df2["rank"])*0.5
mean_ranks = mean_ranks.sort_values()
best_mean_rank = mean_ranks.head(1)[0]
optimized_params = mean_ranks[mean_ranks == best_mean_rank].index.values
print(method+"\tbest mean rank:",best_mean_rank, "top-%:", round(best_mean_rank/mean_ranks.shape[0],4)*100)

print("\topt. parameters:\n\t\t"+"\n\t\t".join(optimized_params) )

# perfromance with optimized parameters
opt_perf1 = df1.loc[optimized_params,performance_col].sort_values(ascending= False)[0]
opt_perf2 = df2.loc[optimized_params,performance_col].sort_values(ascending= False)[0]
print("\tperformance w. optimized:\t%s:%.2f\t%s:%.2f"%(ds1,opt_perf1,ds2,opt_perf2))
# best perfromance 
best_perf1 = df1.loc[:,performance_col].sort_values(ascending= False)
best_perf1 = best_perf1[0]
best_param1 =  df1.loc[df1[performance_col]==best_perf1,:].index.values
print("\tbest parameters %s:\t%.2f"%(ds1,best_perf1))
print("\t\t"+"\n\t\t".join(best_param1))

best_perf2 = df2.loc[:,performance_col].sort_values(ascending= False)
best_perf2 = best_perf2[0]
best_param2 =  df2.loc[df2[performance_col]==best_perf2,:].index.values
print("\tbest parameters %s:\t%.2f"%(ds2,best_perf2))
print("\t\t"+"\n\t\t".join(best_param2))

#print(method, df1.shape[0], df2.shape[0])

UnPaSt	best mean rank: 23.5 top-%: 6.12
	opt. parameters:
		bin=kmeans;pval=0.01;direction=UP-DOWN;clust=WGCNA;dch=0.995;ds=3;preClustering=T
	performance w. optimized:	TCGA-BRCA:0.72	METABRIC:0.76
	best parameters TCGA-BRCA:	0.75
		bin=ward;pval=0.01;direction=UP-DOWN;clust=WGCNA;dch=0.95;ds=3;preClustering=T
	best parameters METABRIC:	0.76
		bin=kmeans;pval=0.05;direction=UP-DOWN;clust=WGCNA;dch=0.995;ds=3;preClustering=T
